In [12]:
import pandas as pd
import re
import pymysql
from io import BytesIO
from sqlalchemy import create_engine
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [ ]:
def connection():
        """This methos creates a connection with the  db with pandas"""
        host = 'localhost'
        port=3308
        user = 'house'
        password = 'house'
        db = 'house'
        try:
            connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
            engine = create_engine(connection_string)
        
        except Exception as e:
            print((f"Error creating connection: {e}"))
            raise

        return engine

In [9]:
engine = connection() 

       

In [10]:
df = pd.read_sql_table('house_WareHouse', con=engine)
        

In [11]:
df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType2,HeatingQC,KitchenQual,GarageFinish,GarageQual,GarageCond
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,2.0,2.0,2.0,3.0,5.0,0.0,2.0,1.0,3.0,2.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,2.0,2.0,2.0,1.0,5.0,0.0,3.0,1.0,3.0,2.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,2.0,2.0,2.0,2.0,5.0,0.0,2.0,1.0,3.0,2.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,2.0,3.0,1.0,3.0,5.0,2.0,2.0,2.0,3.0,2.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,2.0,2.0,2.0,0.0,5.0,0.0,2.0,1.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,3.0,2.0,2.0,3.0,5.0,0.0,3.0,1.0,2.0,3.0
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,3.0,2.0,2.0,3.0,4.0,3.0,3.0,2.0,2.0,3.0
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,2.0,3.0,1.0,3.0,5.0,0.0,2.0,1.0,2.0,3.0
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,3.0,3.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,3.0


In [13]:
#robustScaler

def robustScaler(dataframe):
    Robustscaler = RobustScaler()
    Numeric_scaled = Robustscaler.fit_transform(dataframe)
    dataScaled = pd.DataFrame(Numeric_scaled,columns=dataframe.columns)

    return dataScaled

In [24]:
Y = df[['SalePrice']]

In [28]:
X = df.drop(columns=['SalePrice','Id'])

In [30]:
X.isnull().sum()

MSSubClass        0
LotFrontage     259
LotArea           0
OverallQual       0
OverallCond       0
               ... 
HeatingQC         0
KitchenQual       0
GarageFinish      0
GarageQual        0
GarageCond        0
Length: 69, dtype: int64

In [31]:
X = X.fillna(0)

In [32]:
X= robustScaler(X)

In [33]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [34]:
# Generate a instance of the model RandomForestRegressor
model = RandomForestRegressor(n_estimators=200,max_depth=15, random_state=42,max_features='sqrt',n_jobs=-1,oob_score=True)


In [35]:
# Train the model
model.fit(X_train,Y_train.values.ravel())

RandomForestRegressor(max_depth=15, max_features='sqrt', n_estimators=200,
                      n_jobs=-1, oob_score=True, random_state=42)

In [36]:
# Predict the test set results
Y_test_pred = model.predict(X_test)

In [37]:
# Evaluate the model
mse = mean_squared_error(Y_test, Y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_test_pred)
r2 = r2_score(Y_test, Y_test_pred)

# Mostrar métricas
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.2f}")

MSE: 444367268.31
RMSE: 21080.02
MAE: 10730.38
R^2: 0.94


In [38]:
Y.mean()

SalePrice    180921.19589
dtype: float64